In [1]:
# data downloading
import data_util
import pandas as pd
from collections import defaultdict
import os
import sys

os.chdir("/home/yxjiang/source/ml_playground")
print(os.getcwd())

from util import config

/home/yxjiang/source/ml_playground


In [2]:
dataset_url="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dest_dir = "/tmp/data"
dataset_folder_path = os.path.join(dest_dir, "aclImdb")
data_util.download_data(url=dataset_url, dest_dir=dest_dir)

# generate word to id mapping
word_to_id, word_list = data_util.get_vocabulary(folder_path=dataset_folder_path, file_suffix="vocab")
print("There size of vocabulary is :", len(word_to_id))

Destination folder [/tmp/data] exists.
Target file [aclImdb_v1.tar.gz] exists, skip downloading.
Start to extract [/tmp/data/aclImdb_v1.tar.gz] to [/tmp/data]...
File extracted
Processing vocabulary from [/tmp/data/aclImdb].
There size of vocabulary is : 89527


In [8]:
# model
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, config, vocabulary_size):
        super().__init__()
        self.config = config
        self.rnn_type = config.rnn_type
        self.embed = nn.Embedding(vocabulary_size, config.word_embedding_length)
        self.num_layers = config.num_layers if config.num_layers is not None else 1
        self.directions = config.num_directions if config.num_directions is not None else 1
        self.hidden_size = 128
        if config.rnn_type is nn.RNN:
            self.rnn = nn.RNN(
                input_size=config.word_embedding_length, 
                hidden_size=self.hidden_size, 
                num_layers=self.num_layers,
                bidirectional=False if self.directions == 1 else True
            )
        elif config.rnn_type is nn.LSTM:
            self.cell_size = 128
            self.rnn = nn.LSTM(
                input_size=config.word_embedding_length,
                hidden_size=self.hidden_size,
                num_layers=self.num_layers,
                bidirectional=False if self.directions == 1 else True
            )
        elif config.rnn_type is nn.GRU:
            self.rnn = nn.GRU(
                input_size=config.word_embedding_length, 
                hidden_size=self.hidden_size, 
                num_layers=self.num_layers,
                bidirectional=False if self.directions == 1 else True
            )
        self.fc1 = nn.Linear(in_features=self.hidden_size * self.directions, out_features=64)
        self.dropout = nn.Dropout(config.dropout)
        self.fc2 = nn.Linear(in_features=64, out_features=config.num_classes)

    def forward(self, x):
        batch = x.shape[0]
        x = self.embed(x)  # (batch, sentence_length, embedding_dim)
        x = x.permute(1, 0, 2).contiguous()  # (sentence_length, batch, embedding_dim)

        h0 = torch.zeros((self.num_layers * self.directions, batch, self.hidden_size)).to(self.config.device)
        if self.rnn_type is nn.RNN or self.rnn_type is nn.GRU:
            output, ht = self.rnn(x, h0)
        elif self.rnn_type is nn.LSTM:
            c0 = torch.zeros((self.num_layers * self.directions, batch, self.hidden_size)).to(self.config.device)
            output, (ht, ct) = self.rnn(x, (h0, c0))
        # decompose layers and directions, (layer, directions, batch, embedding_dim)
        ht = ht.view(self.num_layers, self.directions, ht.shape[1], -1) 
        ht = ht[-1]  # get last layer and remove layer dimension, (directions, batch, embedding_dim)
        ht = ht.permute(1, 0, 2)  # (batch, directions, embedding_dim)
        ht = ht.contiguous().view(batch, -1)
        x = F.relu(self.fc1(ht))
        x = self.dropout(x)
        x = self.fc2(x)
        x = x.view(batch, -1)
        return x

In [9]:
# put everything together
import time
from data_util import *
from models import *
from trainer import classification_trainer
from util import config

import torch.optim as optim
import torch.nn.functional as F

# config = config.Config(
#             criteria=nn.CrossEntropyLoss, optimizer=optim.Adam, lr=0.00003, epochs=1000, 
#             batch_size=1024, num_classes=2, sentence_max_length=30, word_embedding_length=128, 
#             activation=F.relu, dropout=0.1, conv_layer_sizes=[3,4,5,6,7]
#         )

def run(rnn_type, **kwargs):
    cfg = config.Config(
        rnn_type=rnn_type,
        criteria=nn.CrossEntropyLoss, optimizer=optim.Adam, lr=0.00002, epochs=200, 
        batch_size=1024, num_classes=2, sentence_max_length=40, word_embedding_length=128, 
        activation=F.relu, dropout=0.1, **kwargs
    )

    pos_train_data_folder = os.path.join(dataset_folder_path, "train/pos")
    neg_train_data_folder = os.path.join(dataset_folder_path, "train/neg")
    train_dataset = MovieReviewDataset(cfg, pos_train_data_folder, neg_train_data_folder, word_to_id, 
                                transform=transforms.Compose([
                                    CutOrPadTransform(cfg), WordsToIdsTransform(cfg, word_to_id)
                                ]))

    pos_test_data_folder = os.path.join(dataset_folder_path, "test/pos")
    neg_test_data_folder = os.path.join(dataset_folder_path, "test/neg")
    test_dataset = MovieReviewDataset(cfg, pos_test_data_folder, neg_test_data_folder, word_to_id, 
                                transform=transforms.Compose([
                                    CutOrPadTransform(cfg), WordsToIdsTransform(cfg, word_to_id)
                                ]))

    train_dataloader = DataLoader(dataset=train_dataset, batch_size=cfg.batch_size, shuffle=True)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=cfg.batch_size)

    # model = TextCNN(cfg, len(word_to_id)).to(device)
    model = RNN(cfg, len(word_list)).to(cfg.device)

    classification_trainer.train(model=model, config=cfg, train_dataloader=train_dataloader, test_dataloader=test_dataloader, check_interval=50)

run(rnn_type=nn.GRU, num_layers=3, num_directions=2)
run(rnn_type=nn.GRU, num_layers=1, num_directions=1)
run(nn.LSTM, num_layers=3, num_directions=2)
run(nn.LSTM, num_layers=1)

[5 seconds](epoch: 0/300)[25000 samples] loss: 0.693.
eval loss: 0.693, accuracy: 51.484% [12871/25000]
[298 seconds](epoch: 50/300)[1275000 samples] loss: 0.466.
eval loss: 0.594, accuracy: 69.644% [17411/25000]
[592 seconds](epoch: 100/300)[2525000 samples] loss: 0.315.
eval loss: 0.697, accuracy: 70.076% [17519/25000]
[884 seconds](epoch: 150/300)[3775000 samples] loss: 0.177.
eval loss: 0.946, accuracy: 69.580% [17395/25000]
[1178 seconds](epoch: 200/300)[5025000 samples] loss: 0.086.
eval loss: 1.381, accuracy: 68.760% [17190/25000]


KeyboardInterrupt: 